<a href="https://colab.research.google.com/github/alejandra2826/Alejandra-Trujillo/blob/main/Practica_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PRACTICA 4**

* **Nombre:** Alejandra Elizabeth Trujillo Navarro
* **e-mail:** alejandra.trujillo2826@alumnos.udg.mx

 # **MODULES**

In [ ]:
import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math

#**CLASSES**

In [ ]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

* Funciones que generen trayectorias tipo Brownian Motion (BM)
* Correlated Random
* Walk (CRW) y Lévy Flight (LF).

In [ ]:
######################################################################
# Brownian Motion Trajectory
######################################################################
def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
  """
  Arguments:
    n_steps: number of steps the Brownian Trajectory will take -> int
    speed: speed of the trajectory or step size -> int
    s_pos: initial position -> [x,y] list
  Returns:
    BM_2d_df: DataFrame with x,y points of the full trajectory
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

In [ ]:
# Correlated Random Walk
def crw(n_steps=1000, velocidad=5, s_pos=[0, 0], exponente_CRW=0.5):

    CRW_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
    CRW_df = pd.concat([CRW_df, temp_df], ignore_index=True)

    # Inicializar el vector de velocidad
    velocidad_vector = np.array([velocidad, 0])

    for i in range(n_steps - 1):
        angulo_giro = np.random.standard_cauchy() * exponente_CRW
        matriz_rotacion = np.array([[np.cos(angulo_giro), -np.sin(angulo_giro)],
                                    [np.sin(angulo_giro), np.cos(angulo_giro)]])
        velocidad_vector = np.dot(matriz_rotacion, velocidad_vector)

        # Actualizar posición
        nueva_posicion = np.array([CRW_df.x_pos[i] + velocidad_vector[0], CRW_df.y_pos[i] + velocidad_vector[1]])
        temp_df = pd.DataFrame([{'x_pos': nueva_posicion[0], 'y_pos': nueva_posicion[1]}])
        CRW_df = pd.concat([CRW_df, temp_df], ignore_index=True)

    return CRW_df


In [ ]:
def levy_flight_alternative(n_steps=100, init_speed=10, start_pos=[0, 0], crw_exponent=0.5, alpha=1.5, beta=0, loc=3.0):
    levy_df = pd.DataFrame(columns=['x_position', 'y_position'])
    temp_df = pd.DataFrame([{'x_position': start_pos[0], 'y_position': start_pos[1]}])
    levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

    for i in range(n_steps - 1):
        # Select turn angle and step size
        turn_angle = np.random.normal(scale=crw_exponent)
        step_size = np.random.standard_cauchy() * alpha + loc

        # Update position
        new_position = [levy_df.x_position[i] + np.cos(turn_angle) * step_size,
                        levy_df.y_position[i] + np.sin(turn_angle) * step_size]

        temp_df = pd.DataFrame([{'x_position': new_position[0], 'y_position': new_position[1]}])
        levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

    return levy_df